# Импорт всех библиотек

In [1]:
import json
import math
import os
import time
from functools import partial
from urllib.parse import urlencode

import faiss
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import requests
from catboost import CatBoostRanker, Pool, CatBoostClassifier
from faiss import read_index, write_index
from sentence_transformers import SentenceTransformer

# Подгружаем модель "ranker.ckpt"

In [2]:
model = CatBoostRanker()
model.load_model("ranker.ckpt")

# Загружаем индексы faiss

In [3]:
index = read_index("candidates.index")
index.is_trained, index.ntotal

(True, 7000000)

Подгружаем "ind2videoid.json"

In [20]:
with open("ind2videoid.json", "r") as f:
    ind2videoid = json.load(f)

# Загружаем модель SentenceTransformer

In [56]:
st_model = SentenceTransformer("cointegrated/rubert-tiny2", device="cuda")

In [26]:
videos_df = pd.read_parquet("videos.parquet", engine="fastparquet", columns=["video_id", "video_title"])

In [58]:
%%time
def search_videos(query, topk=5):
    query_embedding = st_model.encode([query], show_progress_bar=True)[0]
    _, faiss_ind = index.search(np.array([query_embedding]), topk)
    
    videos_ids = [ind2videoid[f"{id}"] for id in faiss_ind[0]]
    
    videos_info = videos_df[videos_df["video_id"].isin(videos_ids)]
    
    return videos_info

query_to_search = "Матч Барселона Россия"
videos = search_videos(query_to_search, topk=5)

print(videos)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

                video_id                                    video_title
13640049  video_10656562             'Матч ТВ' о матче Россия - Румыния
27883696   video_6126551  Обзор матча «Бразилия - Россия». Игра Неймара
28347736   video_5254261                       ГОЛЫ МАТЧА РОССИЯ-ШВЕЦИЯ
28842623  video_16133201        О перфомансе на матче Россия — Бразилия
32401786   video_9424396             Матч Аргентина Россия в Ставрополе
CPU times: user 4.72 s, sys: 174 ms, total: 4.9 s
Wall time: 4.67 s
